In [2]:
pip install ncps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 799.1 kB/s eta 0:00:00


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Reshape, BatchNormalization, Dense,LSTM, Flatten
from sklearn.model_selection import train_test_split
from scipy import stats
from ncps.tf import LTC
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

print("Mounting Google Drive... Done.")

# Function to scale input data
def scaled_in(matrix_spec):
    "Global scaling applied to noisy voice spectrograms (scale between -1 and 1)"
    matrix_spec = (matrix_spec + 46) / 50
    return matrix_spec

print("Defining scaled_in function... Done.")

# Function to scale output data
def scaled_ou(matrix_spec):
    "Global scaling applied to noise models spectrograms (scale between -1 and 1)"
    matrix_spec = (matrix_spec - 6) / 82
    return matrix_spec

print("Defining scaled_ou function... Done.")

# Load data
path_save_spectrogram = '/content/drive/MyDrive/npy/'  # Specify the path to the spectrogram directory

print("Loading data...")
X_in = np.load(os.path.join(path_save_spectrogram, 'noisy_voice_amp_db.npy'))
X_ou = np.load(os.path.join(path_save_spectrogram, 'voice_amp_db.npy'))
print("Data loaded successfully.")

# Model of noise to predict
X_ou = X_in - X_ou

# Check distribution
print("Checking distribution of input data...")
print(stats.describe(X_in.reshape(-1, 1)))
print(stats.describe(X_ou.reshape(-1, 1)))
print("Distribution check completed.")

# Scale input and output data
print("Scaling input and output data...")
X_in = scaled_in(X_in)
X_ou = scaled_ou(X_ou)
print("Data scaled successfully.")

# Check shape of spectrograms
print("Checking shape of spectrograms...")
print("Input shape:", X_in.shape)
print("Output shape:", X_ou.shape)
print("Shape check completed.")


print("Data reshaped successfully.")
import numpy as np
import tensorflow as tf
# Split data into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(X_in, X_ou, test_size=0.10, random_state=42)
y_train_reshaped =  np.reshape(y_train, (450, -1))
y_val_reshaped = np.reshape(y_val, (50, -1))

print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)
print(y_train_reshaped.shape, y_val_reshaped.shape)
print("Data split completed.")

# Load the saved UNet model
# No UNet model in this case, since we're only using RNNs

# Define RNN model
print("Defining RNN model...")
ncp = LTC(32, 16)
rnn_model = tf.keras.Sequential([
    Reshape((-1, 1)),              # Reshape to add a time dimension
    ncp,
    BatchNormalization(),
    Dense(128 * 128 * 1, activation='softmax')
    #Reshape((128, -1)),  # Reshape output of NCP layer to match LSTM input shape
    #LSTM(128, return_sequences=True),# Flatten the output
    #LSTM(128, return_sequences=True),  # LSTM layer with 128 units
    #Conv2D(1, kernel_size=(1, 1), activation='softmax')
    #Reshape((-1, 128, 1)),  # Reshape output of Dense layer to match (batch_size, 128, 1)
    #tf.keras.layers.ZeroPadding2D(((0,0), (0,127))),        # Reshape to match desired output shape
])
 # Reshape to match the shape of the true output
print("RNN model defined successfully.")

#print("Shape after NCP layer:", rnn_model.predict(x_train).shape)


# Compile the RNN model
print("Compiling the RNN model...")
rnn_model.compile(optimizer='Adam',
                  loss='mean_squared_error',  # Use mean squared error loss
                  metrics=['mae'])
print("RNN model compiled successfully.")

# Define filepath for saving the best model
model_checkpoint_path = 'best_model_ltc.keras'

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(model_checkpoint_path,
                             verbose=1,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='auto')

# Train the RNN model
print("Training the RNN model...")
history = rnn_model.fit(x_train, y_train_reshaped, validation_data=(x_val, y_val_reshaped), epochs=20,
                        batch_size=5, verbose=1, callbacks=[checkpoint])
print("Training completed.")

# Save model architecture to JSON file
print("Saving model architecture to JSON file...")
model_json = rnn_model.to_json()
with open('model_ltc.json', 'w') as json_file:
    json_file.write(model_json)
print("Model architecture saved successfully.")

# Save model weights
print("Saving model weights...")
rnn_model.save_weights('model_rnn_weights.keras')
print("Model weights saved successfully.")

# Plot training and validation loss
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Print epoch information and other variables
print("Epochs:", epochs)
print("Training Loss:", loss)
print("Validation Loss:", val_loss)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounting Google Drive... Done.
Defining scaled_in function... Done.
Defining scaled_ou function... Done.
Loading data...
Data loaded successfully.
Checking distribution of input data...
DescribeResult(nobs=8192000, minmax=(array([-80.]), array([0.])), mean=array([-36.32202353]), variance=array([173.18573026]), skewness=array([-0.62897657]), kurtosis=array([1.16821915]))
DescribeResult(nobs=8192000, minmax=(array([-63.09787973]), array([77.94489739])), mean=array([0.36134002]), variance=array([96.05843834]), skewness=array([0.62643615]), kurtosis=array([2.30614789]))
Distribution check completed.
Scaling input and output data...
Data scaled successfully.
Checking shape of spectrograms...
Input shape: (500, 128, 128)
Output shape: (500, 128, 128)
Shape check completed.
Data reshaped successfully.
(450, 128, 128) (50, 128, 128) (450, 128, 128) (50, 128, 128)
(45